## Chat機能

### Chat機能の作成

In [ ]:
import streamlit as st
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from dotenv import load_dotenv

load_dotenv()  # .envファイルからAPIキーを呼び出す。

st.title('RAG チャットボット')

def reset_st_session():
    if 'messages' not in st.session_state:
        st.session_state.messages = []
    st.session_state.model = ChatOpenAI(model='gpt-4o-mini') 

if "session_reset_done" not in st.session_state.keys():
    reset_st_session()
    st.session_state.session_reset_done = True
    
for message in st.session_state.messages:
    with st.chat_message(message['role']):
        st.markdown(message['content'])
        
user_prompt = st.chat_input()
if user_prompt:
    with st.chat_message('user'):
        st.markdown(user_prompt)
    st.session_state.messages.append({'role':'user', 'content':user_prompt})
    
    with st.chat_message('assistant'):
        response = st.write_stream(st.session_state.model.stream(user_prompt))
    st.session_state.messages.append({'role':'assistant', 'content':response})

### 会話履歴を用いたChatの作成

変更点
- `format_prompt()`関数の定義
- `format_prompt()`関数を用いたuser promptの整形とLLMへの入力

In [ ]:
import streamlit as st
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from typing import List, Dict

load_dotenv()  # .envファイルからAPIキーを呼び出す。

st.title('RAG チャットボット')

def reset_st_session():
    if 'messages' not in st.session_state:
        st.session_state.messages = []
    st.session_state.model = ChatOpenAI(model='gpt-4o-mini') 

if "session_reset_done" not in st.session_state.keys():
    reset_st_session()
    st.session_state.session_reset_done = True
    
# prompt整形関数の定義
def format_prompt(query:str, chat_history:List[Dict[str, str]])->str:
    PROMPT = """
    You are a helpful assistant. Answer the user given the chat history:
    
    chat history: {CHAT_HISTORY}
    user query: {QUERY}
    """
    prompt = ChatPromptTemplate.from_template(PROMPT)
    chat_history = '\n\n'.join([f"{message['role']}: {message['content']}" for message in chat_history])
    prompt = prompt.format(CHAT_HISTORY=chat_history,QUERY=query)
    return prompt

    
    
for message in st.session_state.messages:
    with st.chat_message(message['role']):
        st.markdown(message['content'])
        
user_prompt = st.chat_input()
if user_prompt:
    with st.chat_message('user'):
        st.markdown(user_prompt)
    st.session_state.messages.append({'role':'user', 'content':user_prompt})
    
    # ユーザーpromptの整形
    final_prompt = format_prompt(user_prompt,st.session_state.messages)
    
    with st.chat_message('assistant'):
        # 整形されたpromptの入力
        response = st.write_stream(st.session_state.model.stream(final_prompt))
    st.session_state.messages.append({'role':'assistant', 'content':response})

## ファイルの管理機能

### ファイルのアップロード

In [ ]:
import streamlit as st
import os
from app import reset_st_session

st.write('## ファイル管理画面')

if uploaded_files := st.file_uploader("ファイルをアップロード：",accept_multiple_files=True):
    if not os.path.exists("documents"):
        os.makedirs("documents")
    for uploaded_file in uploaded_files:
        with open(os.path.join("documents",uploaded_file.name),"wb") as f:
            f.write(uploaded_file.getbuffer())
    reset_st_session()    
    msg = 'PDFデータの更新が完了しました。'
    st.session_state.messages.append({'role':'assistant', 'content':msg})


### ファイルの削除

変更点
- ファイルの一覧の追加
- 選択したファイルの削除機能の追加

In [ ]:
import streamlit as st
import os
from app import reset_st_session

st.write('## ファイル管理画面')

if uploaded_files := st.file_uploader("ファイルをアップロード：",accept_multiple_files=True):
    if not os.path.exists("documents"):
        os.makedirs("documents")
    for uploaded_file in uploaded_files:
        with open(os.path.join("documents",uploaded_file.name),"wb") as f:
            f.write(uploaded_file.getbuffer())
    reset_st_session()    
    msg = 'PDFデータの更新が完了しました。'
    st.session_state.messages.append({'role':'assistant', 'content':msg})


st.write('#### ファイル一覧')
files_to_remove = []
if os.path.isdir('documents') and len(os.listdir('documents')) != 0:
    for file_name in os.listdir('documents'):
        remove_file = st.checkbox(file_name)
        if remove_file:
            files_to_remove.append(file_name)

if st.button("選択したファイルを削除",key='delete_files'):
    for file in files_to_remove:
        os.remove(f'documents/{file}')
    st.rerun()
    


## RAG機能

### PDFファイルの読み込みと前処理

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
import shutil
import os
from langchain_openai import ChatOpenAI
import chromadb

from typing import List, Dict, Any
from langchain_core.vectorstores import VectorStore
from langchain_core.documents import Document


def load_PDF(path: str) -> List[Document]:
    if os.path.isdir(path):
        loader = PyPDFDirectoryLoader(path, glob="*.pdf")
        documents = loader.load()
    elif os.path.isfile(path):
        if not path.lower().endswith('.pdf'):
            raise ValueError(f"与えられたファイル：  '{path}' はPDFではありません.")
        loader = PyPDFLoader(path)
        documents = loader.load()
    else:
        raise ValueError(f"与えられたパス： '{path}' はファイルでもディレクトリでもありません。")
    if not documents:
        raise ValueError(f"与えられたパス： '{path}' からのファイルの読み込みに失敗しました。")
    return documents

def create_chunks(documents: List[Document], chunk_size: int, chunk_overlap: int) -> List[Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

def init_vector_db(embedding_model:Any,db_path:str)->Chroma:
    if os.path.exists(db_path):
        shutil.rmtree(db_path)
    chromadb.api.client.SharedSystemClient.clear_system_cache()
    vector_db = Chroma(
        collection_name='rag_app_collection',
        embedding_function=embedding_model,
        persist_directory=db_path 
        )
    return vector_db

def get_context_from_db(vector_db:VectorStore, query:str, k:int=5, score_threshold:float=None)->List[Document]:
    contexts = vector_db.similarity_search_with_relevance_scores(query,
                                                                 k=k,
                                                                 score_threshold=score_threshold)
    return contexts

### 重要情報をフォーマットしてpromptに付け加える

※rag.pyに追加（app.pyにあった`format_prompt()`は消す）

In [ ]:
def format_prompt(contexts:List[Document], query:str, chat_history:List[Dict[str, str]])->str:
    PROMPT = """
    You are a helpful assistant. Answer the following questions based on the given context:
    chat history: {CHAT_HISTORY}

    context: {CONTEXT}

    Answer the following questions based on the given context:
    query: {QUERY}
    """
    prompt = ChatPromptTemplate.from_template(PROMPT)
    chat_history = '\n\n'.join([f"{message['role']}: {message['content']}" for message in chat_history])
    sources = [ {'source':doc[0].metadata['source'],'page':doc[0].metadata['page']} for doc in contexts ]
    contexts = '\n'.join([f"CONTEXT {idx}:\n{res.page_content}" for idx, (res, _score) in enumerate(contexts)])
    prompt = prompt.format(CHAT_HISTORY=chat_history,CONTEXT=contexts, QUERY=query)
    return prompt, sources


## RAG Chat機能

### モデル入力にcontext情報を加える